In [1213]:
pip install lxml

In [1214]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import lxml
import re

## Meeting Plan for Nicolas Ouellet and Benedicte Knudson:
We colaborated over Zoom and in person **three** times and used a shared Github Repository that can be found [here](https://github.com/NickOuellet/Fly-Me-To-The-Moon). <br />
We met: <br />
10/11: 5-7pm <br />
10/20: 10-12pm <br />
10/21: 8-11pm <br />


## Step 1 - Scrape SpaceWeatherLive.com

In [1215]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
r = requests.get("https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares.html",timeout=10,headers=headers)
r.status_code

200

In [1216]:
text_response = r.content #https://stackoverflow.com/questions/52389692/beautifulsoup-and-prettify-function
soup = BeautifulSoup(text_response,"html.parser")

In [1217]:
table = soup.findAll("table") #find the table in the HTML

In [1218]:
solar_flare_df = pd.read_html(str(table))

In [1219]:
solfla_df = solar_flare_df[0]#works when we just select the first table

In [1220]:
solfla_df.columns = ["rank", "x_class", "date", "region", "start_time", "maximum_time", "end_time", "movie"] #rename columns to be more descriptive

In [1221]:
solfla_df.head()

,rank,x_class,date,region,start_time,maximum_time,end_time,movie
0,1,X28+,2003/11/04,486,19:29,19:53,20:06,MovieView archive
1,2,X20+,2001/04/02,9393,21:32,21:51,22:03,MovieView archive
2,3,X17.2+,2003/10/28,486,09:51,11:10,11:24,MovieView archive
3,4,X17+,2005/09/07,808,17:17,17:40,18:03,MovieView archive
4,5,X14.4,2001/04/15,9415,13:19,13:50,13:55,MovieView archive


## Step 2 - Tidy the Top 50 Solar Flare Data

In [1222]:
solfla_df = solfla_df[["rank", "x_class", "date", "start_time", "maximum_time", "end_time","region"]]
solfla_df.head()

,rank,x_class,date,start_time,maximum_time,end_time,region
0,1,X28+,2003/11/04,19:29,19:53,20:06,486
1,2,X20+,2001/04/02,21:32,21:51,22:03,9393
2,3,X17.2+,2003/10/28,09:51,11:10,11:24,486
3,4,X17+,2005/09/07,17:17,17:40,18:03,808
4,5,X14.4,2001/04/15,13:19,13:50,13:55,9415


In [1223]:
import datetime

In [1224]:

solfla_df["start_time"] = pd.to_datetime(solfla_df["date"] + " " + solfla_df["start_time"])
solfla_df["maximum_time"] = pd.to_datetime(solfla_df["date"] + " " + solfla_df["maximum_time"])
solfla_df["end_time"] = pd.to_datetime(solfla_df["date"] + " " + solfla_df["end_time"])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [1225]:
solfla_df.drop(columns="date",inplace=True) #drop date table

solfla_df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,rank,x_class,start_time,maximum_time,end_time,region
0,1,X28+,2003-11-04 19:29:00,2003-11-04 19:53:00,2003-11-04 20:06:00,486
1,2,X20+,2001-04-02 21:32:00,2001-04-02 21:51:00,2001-04-02 22:03:00,9393
2,3,X17.2+,2003-10-28 09:51:00,2003-10-28 11:10:00,2003-10-28 11:24:00,486
3,4,X17+,2005-09-07 17:17:00,2005-09-07 17:40:00,2005-09-07 18:03:00,808
4,5,X14.4,2001-04-15 13:19:00,2001-04-15 13:50:00,2001-04-15 13:55:00,9415


In [1226]:
solfla_df.dtypes

rank                     int64
x_class                 object
start_time      datetime64[ns]
maximum_time    datetime64[ns]
end_time        datetime64[ns]
region                   int64
dtype: object

## Step 3 - Scrape the NASA Data

In [1227]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
r = requests.get("https://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html",timeout=10,headers=headers)
r.status_code

200

In [1228]:
text_response = r.content #https://stackoverflow.com/questions/52389692/beautifulsoup-and-prettify-function
soup = BeautifulSoup(text_response,"html.parser")
table = soup.get_text()
split_table = table.splitlines()
split_table = split_table[15:]
split_table = split_table[0:len(split_table)-2] #clean up table by removing top and bottom portion


In [1229]:
nasa_df = pd.DataFrame( #create empty DF 
    columns = ["start_date","start_time","end_date","end_time","start_freq","end_freq","location","region","xray_importance","cme_date","cme_time","central_pos_angle","cpa_width","cme_speed","PHTX" ]
    )
for i,v in enumerate(split_table): 
  row_vals = re.findall(r"\S+",v)
  nasa_df.loc[i] = row_vals[0:15] #assign values for each row from list after splitting string into a list var, also drop 16th entry because sometimes it is useless notation

In [1230]:
nasa_df.drop(columns=["PHTX"],inplace=True) #drop links column

In [1231]:
nasa_df.head()

,start_date,start_time,end_date,end_time,start_freq,end_freq,location,region,xray_importance,cme_date,cme_time,central_pos_angle,cpa_width,cme_speed
0,1997/04/01,14:00,04/01,14:15,8000,4000,S25E16,8026,M1.3,04/01,15:18,74,79,312
1,1997/04/07,14:30,04/07,17:30,11000,1000,S28E19,8027,C6.8,04/07,14:27,Halo,360,878
2,1997/05/12,05:15,05/14,16:00,12000,80,N21W08,8038,C1.3,05/12,05:30,Halo,360,464
3,1997/05/21,20:20,05/21,22:00,5000,500,N05W12,8040,M1.3,05/21,21:00,263,165,296
4,1997/09/23,21:53,09/23,22:16,6000,2000,S29E25,8088,C1.4,09/23,22:02,133,155,712


## Step 4 - Tidy the Nasa Data

**Replace all empty values with NaNs**

In [1232]:
nasa_df.region.replace("-----", np.nan,inplace=True)
nasa_df.xray_importance.replace("----", np.nan,inplace=True)
nasa_df.cme_date.replace("--/--",np.nan, inplace=True)
nasa_df.cme_time.replace("--:--",np.nan,inplace=True)
nasa_df.central_pos_angle.replace("----", np.nan,inplace=True)
nasa_df.cpa_width.replace("----", np.nan,inplace=True)
nasa_df.cpa_width.replace("---", np.nan,inplace=True)
nasa_df.cpa_width.replace("360h", "360",inplace=True)
nasa_df.cme_speed.replace("----", np.nan,inplace=True)
nasa_df.start_freq.replace("????",np.nan,inplace=True)
nasa_df.end_freq.replace("????",np.nan,inplace=True)



**Replace all lower case "Back" values with upper case "BACK" in location column**

In [1233]:
nasa_df.location.replace("Back","BACK",inplace=True)

**Replace all DSF values with FILA in region column**

In [1234]:
nasa_df.region.replace("DSF","FILE",inplace=True)

**Create column to indicate HALO flare, then turn Halo entries into NaN.**<br/>
Halo = 1 <br/>
Non-Halo = 0

In [1235]:
column_vals = []
for index,value in nasa_df["central_pos_angle"].iteritems():
  if value == "Halo":
    column_vals.append(1)
  else:
    column_vals.append(0)
nasa_df["is_halo"] = column_vals

In [1236]:
nasa_df.central_pos_angle.replace("Halo", np.nan,inplace=True)

**Indicate which cpa_widths are lower bounds, make a new column with the info, then remove the non-numeric characters**

In [1237]:
column_vals = []
for index,value in nasa_df["cpa_width"].iteritems():
  if pd.isnull(value) == True:
    column_vals.append(0)
  elif ">" in value:
        column_vals.append(1)
  else:
    column_vals.append(0)
nasa_df["lower_bound"] = column_vals

In [1238]:
def remove_sign(str):
  if pd.isnull(str) == True:
    return str
  return re.sub(">","",str)

nasa_df["cpa_width"] = nasa_df["cpa_width"].apply(remove_sign)

**Reformat date columns and drop repetitive data**

In [1239]:
nasa_df.replace("24:00","23:59",inplace=True)

In [1240]:
for index,value in nasa_df["end_date"].iteritems():
  year = nasa_df.start_date.loc[index][0:4]
  nasa_df["end_date"].loc[index] = pd.to_datetime(year + "/" + nasa_df["end_date"].loc[index] + " " + nasa_df["end_time"].loc[index])

for index,value in nasa_df["cme_date"].iteritems():
  year = nasa_df.start_date.loc[index][0:4]
  if pd.isnull(nasa_df["cme_date"].loc[index]):
    pass
  else:
    nasa_df["cme_date"].loc[index] = pd.to_datetime(year + "/" + nasa_df["cme_date"].loc[index] + " " + nasa_df["cme_time"].loc[index])

nasa_df["start_date"] = pd.to_datetime(nasa_df["start_date"] + " " + nasa_df["start_time"])
nasa_df["end_date"] = pd.to_datetime(nasa_df["end_date"])
nasa_df["cme_date"] = pd.to_datetime(nasa_df["cme_date"])


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [1241]:
nasa_df.rename(columns={"start_date":"start_datetime", "end_date":"end_datetime","cme_date":"cme_datetime"},inplace=True) #rename columns

In [1242]:
nasa_df.drop(columns=["start_time","end_time","cme_time"],inplace=True) #drop unnessecary values

In [1243]:
nasa_df.start_freq = nasa_df.start_freq.astype('float') #change all str types to float
nasa_df.end_freq = nasa_df.end_freq.astype('float')
nasa_df.central_pos_angle = nasa_df.central_pos_angle.astype('float')
nasa_df.cme_speed = nasa_df.cme_speed.astype('float')
nasa_df.cpa_width = nasa_df.cpa_width.astype('float')

In [1244]:
nasa_df.dtypes

start_datetime       datetime64[ns]
end_datetime         datetime64[ns]
start_freq                  float64
end_freq                    float64
location                     object
region                       object
xray_importance              object
cme_datetime         datetime64[ns]
central_pos_angle           float64
cpa_width                   float64
cme_speed                   float64
is_halo                       int64
lower_bound                   int64
dtype: object

In [1245]:
nasa_df.head()

,start_datetime,end_datetime,start_freq,end_freq,location,region,xray_importance,cme_datetime,central_pos_angle,cpa_width,cme_speed,is_halo,lower_bound
0,1997-04-01 14:00:00,1997-04-01 14:15:00,8000.0,4000.0,S25E16,8026,M1.3,1997-04-01 15:18:00,74.0,79.0,312.0,0,0
1,1997-04-07 14:30:00,1997-04-07 17:30:00,11000.0,1000.0,S28E19,8027,C6.8,1997-04-07 14:27:00,NaN,360.0,878.0,1,0
2,1997-05-12 05:15:00,1997-05-14 16:00:00,12000.0,80.0,N21W08,8038,C1.3,1997-05-12 05:30:00,NaN,360.0,464.0,1,0
3,1997-05-21 20:20:00,1997-05-21 22:00:00,5000.0,500.0,N05W12,8040,M1.3,1997-05-21 21:00:00,263.0,165.0,296.0,0,0
4,1997-09-23 21:53:00,1997-09-23 22:16:00,6000.0,2000.0,S29E25,8088,C1.4,1997-09-23 22:02:00,133.0,155.0,712.0,0,0
